In [ ]:
import copy
import os
import numpy as np
from numpy.linalg import norm

target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\"
os.chdir(target_dir)

import processing.readwritedatafiles as readwritedatafiles

In [ ]:
%%capture
%run "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\quail" "conduit.py"

# Run Quail (line magics must be first)

In [ ]:
# Load a solver and interact
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_0.pkl")
# dir(solver.physics)
# solver.get_boundary_face_residuals(self, U, res)
# res =  np.zeros_like(solver.state_coeffs)
# solver.get_interior_face_residuals(solver.state_coeffs, res)
# solver.get_boundary_face_residuals(solver.state_coeffs, res)
# solver_tools.mult_inv_mass_matrix(mesh, solver, self.dt, res)

In [ ]:
s0 = copy.deepcopy(solver)
s0.state_coeffs[:, :, s0.physics.get_momentum_slice()] = 0.0
for i in range (5):
  r = s0.stepper.take_time_step(s0)
  s0.state_coeffs[:, :, s0.physics.get_momentum_slice()] = 0.0
  print(norm(r))

In [ ]:
# Extract p in FE context
p = s0.physics.get_conv_flux_interior(s0.state_coeffs)[0][:,:,s0.physics.get_momentum_slice()]

In [ ]:
# Residual evaluation
res =  np.zeros_like(s0.state_coeffs)
s0.get_element_residuals(s0.state_coeffs, res)
# print(res)

import solver.tools as solver_tools

Uq = s0.state_coeffs
Sq = np.zeros_like(Uq) # [ne, nq, ns]
Sq = s0.physics.eval_source_terms(Uq, s0.elem_helpers.x_elems, s0.time, Sq)
    # [ne, nq, ns]
solver_tools.calculate_source_term_integral(
					s0.elem_helpers, Sq)

print(Sq)

In [ ]:
# [S for S in s0.physics.source_terms if S.name == "GravitySource"] 
# Assume source term 0 is gravity
# dir(s0.physics.source_terms[0].get_jacobian)
# s0.physics.source_terms[0].get_jacobian(
#   s0.physics, s0.state_coeffs, s0.mesh.node_coords, s0.time\
#   )
# s0.mesh.node_coords
# s0.physics.get_state_slice("pDensityWv")

relax_param = 0.01

# R = s0.physics.compute_additional_variable["Pressure"] - s0.physics.source_terms[0]
# p = s0.physics.compute_additional_variable("Pressure", s0.state_coeffs, True)
# S =  -s0.physics.source_terms[0].get_source(
#       s0.physics, s0.state_coeffs, s0.mesh.node_coords, s0.time)

In [ ]:
import numerics.helpers.helpers as helpers
s0 = copy.deepcopy(solver)

p_top = 1e5 # Specify boundary condition at top: pressure = p_atm
def eval_relaxeqn(s0):
# if True:
	''' Compute gradients of hydrostatic equation residual with respect to all states'''
	# Shorthand the state coefficients
	Uq = s0.state_coeffs
	Uq = helpers.evaluate_state(s0.state_coeffs, s0.elem_helpers.basis_val,
				skip_interp=s0.basis.skip_interp)

	# Compute dp/dq (ne, nd, nq)
	dpdq = s0.physics.compute_pressure_sgradient(s0.state_coeffs)
	# Compute (u^2 + p) . grad(phi) == p . grad(phi)
	Fq = s0.physics.get_conv_flux_interior(Uq)[0] # [ne, nq, ns, ndims]
	fdotgradphi = solver_tools.calculate_volume_flux_integral(
						s0, s0.elem_helpers, Fq) # [ne, nb, ns]
	# Compute dp/dq . grad(phi)
	dpdqdotgradphi = solver_tools.calculate_volume_flux_integral(
						s0, s0.elem_helpers, np.expand_dims(dpdq,axis=3)) # [ne, nb, ns]

	interior_res = np.zeros_like(Uq)
	solver.get_interior_face_residuals(Uq, interior_res)
	dinterior_res = dpdq * interior_res

	# Manual hack
	bdry_res = np.zeros_like(Uq)
	solver.get_boundary_face_residuals(Uq, bdry_res)
	# Override bdry pressure
	bdry_res[-1,:,s0.physics.get_state_index("XMomentum")] = -p_top
	dbdry_res = dpdq * bdry_res

	# Compute dS/dq (ne, nd, nq), where S = -rho*g
	dSdq = s0.physics.source_terms[0].get_jacobian(
		s0.physics, s0.state_coeffs, s0.mesh.node_coords, s0.time\
		)[:,:,s0.physics.get_state_index("XMomentum"),:]
	# Compute S . phi == (-rho*g) . phi
	Sq = np.zeros_like(Uq)
	Sq = s0.physics.eval_source_terms(Uq, s0.mesh.node_coords, s0.time, Sq) # [ne, nq, ns]
	Sphi = solver_tools.calculate_source_term_integral(
			s0.elem_helpers, Sq) # [ne, nb, ns]
	# Compute dSdq . phi
	dSphi = solver_tools.calculate_source_term_integral(
			s0.elem_helpers, dSdq) # [ne, nb, ns]

	# Compute objective residual (sum on RHS) across all indices
	R_all_indices = interior_res + fdotgradphi + Sphi + bdry_res
	# Constrain to pressure index
	R = R_all_indices[:,:,s0.physics.get_momentum_slice()]
	# Compute derivative of residual with respect to state
	dR = dpdqdotgradphi + dSphi + dinterior_res + dbdry_res # [ne, nb, ns]

	# Get quadratic problem
	Q = 0.5 * R**2
	dQ = R * dR

	return Q, dQ
# print(bdry_res)

In [ ]:
relax_param = 1e-7
s0 = copy.deepcopy(solver)

Q0, dQ = eval_relaxeqn(s0)
# step = np.zeros_like(dQ)
# step[:,:,s0.physics.get_state_index("pDensityWv")] = dQ[:,:,s0.physics.get_state_index("pDensityWv")]
# step[:,:,s0.physics.get_state_index("pDensityM")] = dQ[:,:,s0.physics.get_state_index("pDensityM")]
# dQ.shape

print(f"{norm(Q0):.3e}")

for i in range(100):
        Q, dQ = eval_relaxeqn(s0)

        # Constrain weight ratio, rather than weight fraction (for now)
        ratio = s0.state_coeffs[:,:,s0.physics.get_state_index("pDensityWv")] \
                / s0.state_coeffs[:,:,s0.physics.get_state_index("pDensityM")]

        # Construct step
        v = np.zeros_like(dQ)
        v[:, :, s0.physics.get_state_index("pDensityWv")] = ratio
        v[:, :, s0.physics.get_state_index("pDensityM")] = 1.0
        v /= norm(v, axis=-1, keepdims=True)

        # Evaluate directional derivative
        DvQ = np.einsum("ijk, ijl -> ij", dQ, v)

        step_size = - relax_param * DvQ
        # Step limiting
        step_guess = np.einsum("ij, ijk -> ijk", step_size, v)[:,:,1:3]
        step_size_factor = np.minimum(0.5*np.ones_like(step_guess),
                (1e-11 + np.abs(step_guess)) / (1e-11 + s0.state_coeffs[:,:,1:3])) \
                * (1e-11 + s0.state_coeffs[:,:,1:3]) / (1e-11 + np.abs(step_guess))
        step_size *= np.min(step_size_factor, axis=2)
        
        # DvQ = (Q.squeeze(axis=2)/np.einsum("ijk, ijl -> ij", dQ, v))
        step_dir = np.einsum("ij, ijk -> ijk", step_size, v)
        
        s0.state_coeffs += step_dir
        Q, dQ = eval_relaxeqn(s0)
        print(f"{norm(Q):.3e}")

# Check ratio constraint
# s0.state_coeffs[:,:,s0.physics.get_state_index("pDensityWv")] \
        # / s0.state_coeffs[:,:,s0.physics.get_state_index("pDensityM")]
# print(s0.state_coeffs)
# step_size

print(Q/Q0)

In [ ]:
s0.state_coeffs

In [ ]:
import numerics.helpers.helpers as helpers
s0 = copy.deepcopy(solver)

p_top = 1e5 # Specify boundary condition at top: pressure = p_atm
def eval_relaxeqn(s0):
# if True:
	''' Compute gradients of hydrostatic equation residual with respect to all states'''
	# Shorthand the state coefficients
	Uq = s0.state_coeffs
	Uq = helpers.evaluate_state(s0.state_coeffs, s0.elem_helpers.basis_val,
				skip_interp=s0.basis.skip_interp)

	# Compute dp/dq (ne, nd, nq)
	dpdq = s0.physics.compute_pressure_sgradient(s0.state_coeffs)
	# Compute (u^2 + p) . grad(phi) == p . grad(phi)
	Fq = s0.physics.get_conv_flux_interior(Uq)[0] # [ne, nq, ns, ndims]
	fdotgradphi = solver_tools.calculate_volume_flux_integral(
						s0, s0.elem_helpers, Fq) # [ne, nb, ns]
	# Compute dp/dq . grad(phi)
	dpdqdotgradphi = solver_tools.calculate_volume_flux_integral(
						s0, s0.elem_helpers, np.expand_dims(dpdq,axis=3)) # [ne, nb, ns]

	interior_res = np.zeros_like(Uq)
	solver.get_interior_face_residuals(Uq, interior_res)
	dinterior_res = dpdq * interior_res

	# Manual hack
	bdry_res = np.zeros_like(Uq)
	solver.get_boundary_face_residuals(Uq, bdry_res)
	# Override bdry pressure
	bdry_res[-1,:,s0.physics.get_state_index("XMomentum")] = -p_top
	dbdry_res = dpdq * bdry_res

	# Compute dS/dq (ne, nd, nq), where S = -rho*g
	dSdq = s0.physics.source_terms[0].get_jacobian(
		s0.physics, s0.state_coeffs, s0.mesh.node_coords, s0.time\
		)[:,:,s0.physics.get_state_index("XMomentum"),:]
	# Compute S . phi == (-rho*g) . phi
	Sq = np.zeros_like(Uq)
	Sq = s0.physics.eval_source_terms(Uq, s0.mesh.node_coords, s0.time, Sq) # [ne, nq, ns]
	Sphi = solver_tools.calculate_source_term_integral(
			s0.elem_helpers, Sq) # [ne, nb, ns]
	# Compute dSdq . phi
	dSphi = solver_tools.calculate_source_term_integral(
			s0.elem_helpers, dSdq) # [ne, nb, ns]

	# Compute objective residual (sum on RHS) across all indices
	R_all_indices = interior_res + fdotgradphi + Sphi + bdry_res
	# Constrain to pressure index
	R = R_all_indices[:,:,s0.physics.get_momentum_slice()]
	# Compute derivative of residual with respect to state
	dR = dpdqdotgradphi + dSphi + dinterior_res + dbdry_res # [ne, nb, ns]

	# Get quadratic problem
	Q = 0.5 * R**2
	dQ = R * dR

	return Q, dQ

In [ ]:
step_dir.shape

In [ ]:
step.shape